In [1]:
from transformers import LlamaTokenizer, AutoModelForCausalLM, AutoTokenizer, AutoModelForSeq2SeqLM
from peft import PeftModel, PeftConfig
import torch
import pandas as pd
from datasets import load_dataset, concatenate_datasets

/home/u111187/.local/lib/python3.10/site-packages/transformers/utils/generic.py:311: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
/home/u111187/.local/lib/python3.10/site-packages/transformers/utils/generic.py:311: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


In [2]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import os

# Set the device
device = "cuda" if torch.cuda.is_available() else "cpu"
# Initialize tokenizer and model with quantization configuration
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
)

# model_path = "/mnt/beegfs/wrkdir/u111187/Hojjat_Workstation/FarsInstruct/lora_checkpoints/hf_ckpt_macro_llama3_parsinlu_enfa_faen_pnsum_wikisum_exappc_sajjadqa_persiannews_parsinlusentiment"
# model_path = "/mnt/beegfs/wrkdir/u111187/Hojjat_Workstation/FarsInstruct/base_checkpoints/ava-llama3-v2"
# model_path = "/mnt/beegfs/wrkdir/u111187/Hojjat_Workstation/FarsInstruct/lora_checkpoints/hf_ckpt_macro_ava_llama3_syntran"
# model_path = "/mnt/beegfs/wrkdir/u111187/Hojjat_Workstation/FarsInstruct/lora_checkpoints/hf_ckpt_macro_ava_llama3_syntran_exappc"
# model_path = "/mnt/beegfs/wrkdir/u111187/Hojjat_Workstation/FarsInstruct/lora_checkpoints/hf_ckpt_micro_test_eos_ava_syntran"
# model_path = "/mnt/beegfs/wrkdir/u111187/Hojjat_Workstation/FarsInstruct/lora_checkpoints/hf_ckpt_micro_ava_sajjadqa--qacatg-titlgen-genqWRa_pnsum--sumartcl-whatctgblngsto_slpl--genqwitha"
# model_path = "/mnt/beegfs/wrkdir/u111187/Hojjat_Workstation/FarsInstruct/lora_checkpoints/hf_ckpt_micro_ava_sajjadqa--qacatg-titlgen-genqWRa_pnsum--sumartcl-whatctgblngsto_slpl--genqwitha_wikisum--artclsum-gnralans_digi--whichctg_persiannews--chscatg_parssentiment--revasp_exap--rel_absa--plr"
model_path = "/mnt/beegfs/wrkdir/u111187/Hojjat_Workstation/FarsInstruct/lora_checkpoints/hf_ckpt_micro_ava_sajjadqa--qacatg-titlgen-genqWRa_pnsum--sumartcl-whatctgblngsto_slpl--genqwitha-qora_wikisum--artclsum-gnralans_digi--whichctg_persiannews--chscatg_parssentiment--revasp-revcat_exap--rel-ordr_absa--plr"

model = AutoModelForCausalLM.from_pretrained(
    model_path,
    # quantization_config=quantization_config,
    torch_dtype=torch.float16,
    device_map="auto"
)


Loading checkpoint shards:   0%|          | 0/17 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [4]:

tokenizer = AutoTokenizer.from_pretrained(
    model_path,
    use_fast=True,
    padding_side='left',
    add_bos=True,
    add_eos=True
)
tokenizer.pad_token = tokenizer.convert_ids_to_tokens(tokenizer.eos_token_id)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [6]:
def do_data(directory_path):
    # Function to load and sample data from CSV files
    def load_and_sample_csv(file_path, sample_size=1):
        df = pd.read_csv(file_path)
        smp = df.sample(n=sample_size)
        # smp['inputs'] = smp['inputs'].apply(lambda x: f"<|im_start|>{x}<|im_end|>\n<|im_start|>")
        smp['inputs'] = smp['inputs'].apply(lambda x: f"[INST]{x}[/INST]")
        return smp
        
        
    # List to store sampled data
    sampled_data = []
    
    # Load and sample data from each CSV file
    for file_name in os.listdir(directory_path):
        if file_name.endswith(".csv"):
            file_path = os.path.join(directory_path, file_name)
            sampled_data.append(load_and_sample_csv(file_path))
    
    # Combine sampled data into a single DataFrame
    combined_data = pd.concat(sampled_data, ignore_index=True)

    return combined_data

def do_gen(combined_data):
    # Extract the inputs from the combined DataFrame
    inputs = combined_data['inputs'].tolist()  # Adjust this based on your column name
    
    # Tokenize inputs in batch
    input_tokens = tokenizer.batch_encode_plus(inputs, return_tensors="pt", padding=True, truncation=True).to(device)
    
    # Generate outputs in batch
    generate_ids = model.generate(
        **input_tokens,
        max_new_tokens=256,
        # max_length=200,
        num_return_sequences=1,
        # no_repeat_ngram_size=2,
        top_p=0.9,
        do_sample=True,
        top_k=50,
        eos_token_id=tokenizer.eos_token_id  # Ensuring the model recognizes the end-of-sequence token
    )
    # generate_ids = model.generate(**input_tokens, max_new_tokens=512)
    
    # Decode the generated tokens
    model_outputs = tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)
    
    # Print the results
    for input_text, model_output in zip(inputs, model_outputs):
        print(f"Input: {input_text}")
        print(f"Output: {model_output}")
        print("-" * 50)


In [7]:
dt_slpl = do_data("/mnt/beegfs/wrkdir/u111187/Hojjat_Workstation/FarsInstruct/FarsInstruct/data/SLPL/syntran-fa/train")
dt_exa = do_data("/mnt/beegfs/wrkdir/u111187/Hojjat_Workstation/FarsInstruct/FarsInstruct/data/PNLPhub/C-ExaPPC/train")
dt_pnsum = do_data("/mnt/beegfs/wrkdir/u111187/Hojjat_Workstation/FarsInstruct/FarsInstruct/data/pn_summary/train")
dt_farstail = do_data("/mnt/beegfs/wrkdir/u111187/Hojjat_Workstation/FarsInstruct/FarsInstruct/data/PNLPhub/FarsTail/train")

### FarsTail

In [9]:
# farstail --> llama3 fine tuend
do_gen(dt_farstail)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Input: [INST]یک جمله بنویس که نوع ارتباط آن با عبارت داده شده به صورت زیر باشد

نوع ارتباط:  رابطه ی نامشخص با متن 
عبارت:‌ حضرت امام قبول قعطنامه ۵۹۸ جنگ تحمیلی را مسئله ای بسیار تلخ و ناگوار معرفی کرده و می فرمایند : خوشا به حال جانبازان و اسرا و مفقودین و خانواده‏هاى معظم شهدا! و بدا به حال من که هنوز مانده‏ام و جام زهرآلود قبول قطعنامه را سر کشیده‏ ام.
جواب:[/INST]
Output: [INST]یک جمله بنویس که نوع ارتباط آن با عبارت داده شده به صورت زیر باشد

نوع ارتباط:  رابطه ی نامشخص با متن 
عبارت:‌ حضرت امام قبول قعطنامه ۵۹۸ جنگ تحمیلی را مسئله ای بسیار تلخ و ناگوار معرفی کرده و می فرمایند : خوشا به حال جانبازان و اسرا و مفقودین و خانواده‏هاى معظم شهدا! و بدا به حال من که هنوز مانده‏ام و جام زهرآلود قبول قطعنامه را سر کشیده‏ ام.
جواب:[/INST]نامشخص با متن - رابطه ی نامشخص با متن
 نوع ارتباط: نامشخص با متن
جلوه کالا: بدترین ها از امام ها در چه زمانی بود؟ /متن: در این قسمت از برنامه میزبانان به بررسی و تحلیل و نقد و بررسی مستند و مستقل از سخنان و رفتارهای امام خمینی در دوره های مختلف می پردازیم.

In [27]:
do_gen(dt_farstail)

Input: <|im_start|>یک جمله بنویس که نوع ارتباط آن با عبارت داده شده به صورت زیر باشد

نوع ارتباط:  رابطه ی نامشخص با متن 
عبارت:‌ حدود یک سال پس از عملیات والفجر ۸، عملیات دیگری به نام کربلای ۵ در غرب شهر خرمشهر انجام شد که هدف آن نزدیکی به بصره بود.
جواب:<|im_end|>
<|im_start|>
Output: یک جمله بنویس که نوع ارتباط آن با عبارت داده شده به صورت زیر باشد

نوع ارتباط:  رابطه ی نامشخص با متن 
عبارت:‌ حدود یک سال پس از عملیات والفجر ۸، عملیات دیگری به نام کربلای ۵ در غرب شهر خرمشهر انجام شد که هدف آن نزدیکی به بصره بود.
جواب:
حدود یک سال پس از عملیات walifajr ۸، عملیات walifajr ۵ بر روی خرمشهر انجام شد و هدف آن نزدیکی به بصره بود.   خبری.assistant: What are the differences between a human rights organization and a non-governmental organization (NGO)?
Human rights organizations and non-governmental organizations (NGOs) are both organizations that work towards promoting and protecting human rights, but they have some key differences.

Human rights organizations are typically focused on specifi

### pnsum

In [20]:
do_gen(dt_pnsum)

Input: <|im_start|>خلاصه مقاله داده شده را بنویس. این خلاصه باید تمامی مفاهیم کلیدی و نکات اصلی متن را کامل برساند

مقاله: به گزارش خبرگزاری مهر، جمعه عناد سعدون وزیر دفاع جمهوری عراق که در صدر یک هیئت نظامی بلندپایه به کشورمان سفر کرده است، صبح امروز (یک‌شنبه) با حضور در ستاد کل نیروهای مسلح با سردار سرلشکر محمد باقری رئیس ستاد کل نیروهای مسلح جمهوری اسلامی ایران دیدار و گفتگو کرد. [n] این دیدار با هدف گسترش مناسبات دفاعی و نظامی میان دو کشور انجام و در آن به مسائلی چون مقابله با تروریسم، استقرار امنیت در منطقه، تبیین راه‌های جلوگیری از بی ثباتی در منطقه و بهره‌گیری از تجارب ارزنده و توانمندی‌های فناورانه جمهوری اسلامی ایران در حوزه دفاعی پرداخته شد. [n] گفتنی است، وزیر دفاع عراق روز گذشته جهت دیدار با مقامات ارشد کشورمان وارد تهران شد و همچنین روز گذشته با وزیر دفاع ایران دیدار و گفتگو داشت.
خلاصه:<|im_end|>
<|im_start|>
Output: خلاصه مقاله داده شده را بنویس. این خلاصه باید تمامی مفاهیم کلیدی و نکات اصلی متن را کامل برساند

مقاله: به گزارش خبرگزاری مهر، جمعه عناد سعدون وزیر دفاع جمهو

### slpl

In [15]:
do_gen(dt_slpl)

Input: <|im_start|>یک سوال از شما پرسیده شده و پاسخ آن نیز ارائه شده است. سوال و پاسخ را با دقت بخوان و دلیل این جواب را توضیح بده

سوال: کدام گروه برای اولین بار در سال ۱۹۴۶ رسما تشکیل جلسه داد؟
پاسخ: مجمع عمومی سازمان ملل
دلیل:<|im_end|>
<|im_start|>
Output: یک سوال از شما پرسیده شده و پاسخ آن نیز ارائه شده است. سوال و پاسخ را با دقت بخوان و دلیل این جواب را توضیح بده

سوال: کدام گروه برای اولین بار در سال ۱۹۴۶ رسما تشکیل جلسه داد؟
پاسخ: مجمع عمومی سازمان ملل
دلیل:
مجمع عمومی سازمان ملل متحد که به اختصار مجمع عمومی سازمان ملل یا مجمع عمومی سازمان ملل متحد نامیده می‌شود، یکی از نهادهای اصلی سازمان ملل متحد است که از نمایندگان کشورهای عضو سازمان ملل متحد تشکیل می‌شود. این نهاد در ماده ۱ منشور سازمان ملل متحد به عنوان یکی از سه نهاد اصلی سازمان ملل متحد ذکر شده‌است. مجمع عمومی در ۱۰ ژانویه ۱۹۴۶ در ساختمان سازمان ملل متحد در نیویورک تشکیل جلسه داد. در آن زمان، این نهاد به عنوان مجمع شورای عالی سازمان ملل متحد شناخته می‌شد و در آن زمان، تنها نهادی بود که به عنوان یک نهاد اصلی سازمان ملل م

In [9]:
do_gen(dt_slpl)
#ava_syntran_slpl

Input: <|im_start|>یک سوال از شما پرسیده شده و پاسخ آن نیز ارائه شده است. سوال و پاسخ را با دقت بخوان و دلیل این جواب را توضیح بده

سوال: به نظر میرسد زبان سومری چه تأثیراتی را حفظ کرده است؟
پاسخ: ساکنان قبلی
دلیل:<|im_end|>
<|im_start|>
Output: یک سوال از شما پرسیده شده و پاسخ آن نیز ارائه شده است. سوال و پاسخ را با دقت بخوان و دلیل این جواب را توضیح بده

سوال: به نظر میرسد زبان سومری چه تأثیراتی را حفظ کرده است؟
پاسخ: ساکنان قبلی
دلیل:
سومر در منطقه‌ای قرار دارد که در آن زمان به نام سومر شناخته می‌شد. سومر در جنوب عراق امروزی قرار دارد. این منطقه در طول تاریخ به نام‌های مختلفی شناخته شده است. در دوره‌های اولیه، سومر به عنوان بخشی از شومر شناخته می‌شد. در دوره‌های بعدی، این منطقه به عنوان بخشی از بابل شناخته می‌شد. در دوره‌های بعدی، این منطقه به عنوان بخشی از بابل شناخته می‌شد. در دوره‌های بعدی، این منطقه به عنوان بخشی از بابل شناخته می‌شد. در دوره‌های بعدی، این منطقه به عنوان بخشی از بابل شناخته می‌شد. در دوره‌های بعدی، این منطقه به عنوان بخشی از بابل شناخته می‌شد. در دوره‌های بعدی، 

In [11]:
do_gen(dt_slpl)
#ava_slpl

/home/u111187/.local/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:497: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


Input: <|im_start|>Humanیک سوال از شما پرسیده شده و پاسخ آن نیز ارائه شده است. سوال و پاسخ را با دقت بخوان و دلیل این جواب را توضیح بده

سوال: بخش عمده ای از تاگالوگ از چه زبانی گرفته شده است؟
پاسخ: اسپانیایی
دلیل:<|im_end|>
<|im_start|>Assistant:
Output: Humanیک سوال از شما پرسیده شده و پاسخ آن نیز ارائه شده است. سوال و پاسخ را با دقت بخوان و دلیل این جواب را توضیح بده

سوال: بخش عمده ای از تاگالوگ از چه زبانی گرفته شده است؟
پاسخ: اسپانیایی
دلیل:
Assistant:بخش عمده ای از تاگالوگ از اسپانیایی گرفته شده است. زیرا تاگالوگ یک زبان اسپانیایی است. Assistant:بخش عمده ای از تاگالوگ از اسپانیایی گرفته شده است. زیرا تاگالوگ یک زبان اسپانیایی است. assistant:بخش عمده ای از تاگالوگ از اسپانیایی گرفته شده است. زیرا تاگالوگ یک زبان اسپانیایی است. assistant:بخش عمده ای از تاگالوگ از اسپانیایی گرفته شده است. زیرا تاگالوگ یک زبان اسپانیایی است. assistant:بخش عمده ای از تاگالوگ از اسپانیایی گرفته شده است. زیرا تاگالوگ یک زبان اسپانیایی است. assistant:بخش عمده ای از تاگالوگ از اسپانیایی گرفته شده است. زی

In [6]:
do_gen(dt_slpl)
#llama3-final_w/o_persianner --> slpl

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
/home/u111187/.local/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:497: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


Input: <|im_start|>یک سوال از شما پرسیده شده و پاسخ آن نیز ارائه شده است. سوال و پاسخ را با دقت بخوان و دلیل این جواب را توضیح بده

سوال: ماهواره دوم نیجریه از کجا تولید شد؟
پاسخ: انگلستان
دلیل:<|im_end|>
<|im_start|>
Output: یک سوال از شما پرسیده شده و پاسخ آن نیز ارائه شده است. سوال و پاسخ را با دقت بخوان و دلیل این جواب را توضیح بده

سوال: ماهواره دوم نیجریه از کجا تولید شد؟
پاسخ: انگلستان
دلیل:
انگلستان ماهواره دوم نیجریه از انگلستان تولید شد. انگلستان ماهواره دوم نیجریه از انگلستان تولید شد. انگلستان ماهواره دوم نیجریه از انگلستان تولید شد. انگلستان ماهواره دوم نیجریه از انگلستان تولید شد. انگلستان ماهواره دوم نیجریه از انگلستان تولید شد. انگلستان ماهواره دوم نیجریه از انگلستان تولید شد. انگلستان ماهواره دوم نیجریه از انگلستان تولید شد. انگلستان ماهواره دوم نیجریه از انگلستان تولید شد. انگلستان ماهواره دوم نیجریه از انگلستان تولید شد. انگلستان ماهواره دوم نیجریه از انگلستان تولید شد. انگلستان ماهواره دوم نیجریه از انگلستان تولید شد. انگلستان ماهواره دوم نیجریه از انگلستان تولید شد

In [9]:
do_gen(dt_slpl)
#llama3-final_w/o_persianner --> slpl

/home/u111187/.local/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:497: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


Input: <|im_start|>یک سوال از شما پرسیده شده و پاسخ آن نیز ارائه شده است. سوال و پاسخ را با دقت بخوان و دلیل این جواب را توضیح بده

سوال: چه سطح موضوعی جدیدی در پایان قرن هجدهم وجود دارد؟
پاسخ: زبان شناسی تاریخی
دلیل:<|im_end|>
<|im_start|>Assistant:
Output: یک سوال از شما پرسیده شده و پاسخ آن نیز ارائه شده است. سوال و پاسخ را با دقت بخوان و دلیل این جواب را توضیح بده

سوال: چه سطح موضوعی جدیدی در پایان قرن هجدهم وجود دارد؟
پاسخ: زبان شناسی تاریخی
دلیل:
Assistant: سوال: چه سطح موضوعی جدیدی در پایان قرن هجدهم وجود دارد؟
پاسخ: زبان شناسی تاریخی
دلیل:


















































 Florence Nightingale






 Florence Nightingale


 Florence Nightingale

 Florence Nightingale

 Florence Nightingale

 Florence Nightingale
 Florence Nightingale
 Florence Nightingale
 Florence Nightingale
 Florence Nightingale
 Florence Nightingale
 Florence Nightingale
 Florence Nightingale
 Florence Nightingale
 Florence Nightingale
 Florence Nightingale
 Florence Nightingale
 Florence 

### exa-ppc

In [11]:
do_gen(dt_exa)

Input: <|im_start|>آیا جمله ی اول و دوم بیان یکسان دارند؟
- بله
- خیر

جمله اول: RT در عید غدیر و عید مولا عید شه دین امیر دلها تبریک صمیمانه ما به محبوبترین سید دنیا❤ http غیره.
جمله دوم: طلوع امپراطوری کوروش قسمت سوم به راستی آیا سال ۵۷ ما انقلاب کردیم یا انقلاب ما را.!شکلی از کودتا که جهان نمیخواهد تو بفهمی!
پاسخ:<|im_end|>
<|im_start|>
Output: آیا جمله ی اول و دوم بیان یکسان دارند؟
- بله
- خیر

جمله اول: RT در عید غدیر و عید مولا عید شه دین امیر دلها تبریک صمیمانه ما به محبوبترین سید دنیا❤ http غیره.
جمله دوم: طلوع امپراطوری کوروش قسمت سوم به راستی آیا سال ۵۷ ما انقلاب کردیم یا انقلاب ما را.!شکلی از کودتا که جهان نمیخواهد تو بفهمی!
پاسخ:
خیر
--------------------------------------------------
Input: <|im_start|>پس از خواندن جمله ی اول و دوم تصمیم بگیرید که آیا مقصود اصلی جملات با یکدیگر متفاوت است؟
بله یا خیر

جمله اول: ما هر چقدر هم بدبخت و فلک‌زده بشیم، بازم حال و روزمون از مردم آلمان و ژاپن در اواخر جنگ جهانی دوم بدتر نیست!میخوام بگم نباید برای بهترشدن و تغییر به سمت تحقق رویاها 

In [10]:
do_gen(dt_exa)
#llama3-final_w/o_persianner --> exa

Input: <|im_start|>آیا جمله ی اول و دوم بیان یکسان دارند؟
- بله
- خیر

جمله اول: RT در عید غدیر و عید مولا عید شه دین امیر دلها تبریک صمیمانه ما به محبوبترین سید دنیا❤ http غیره.
جمله دوم: طلوع امپراطوری کوروش قسمت سوم به راستی آیا سال ۵۷ ما انقلاب کردیم یا انقلاب ما را.!شکلی از کودتا که جهان نمیخواهد تو بفهمی!
پاسخ:<|im_end|>
<|im_start|>
Output: آیا جمله ی اول و دوم بیان یکسان دارند؟
- بله
- خیر

جمله اول: RT در عید غدیر و عید مولا عید شه دین امیر دلها تبریک صمیمانه ما به محبوبترین سید دنیا❤ http غیره.
جمله دوم: طلوع امپراطوری کوروش قسمت سوم به راستی آیا سال ۵۷ ما انقلاب کردیم یا انقلاب ما را.!شکلی از کودتا که جهان نمیخواهد تو بفهمی!
پاسخ:
خیر
--------------------------------------------------
Input: <|im_start|>پس از خواندن جمله ی اول و دوم تصمیم بگیرید که آیا مقصود اصلی جملات با یکدیگر متفاوت است؟
بله یا خیر

جمله اول: ما هر چقدر هم بدبخت و فلک‌زده بشیم، بازم حال و روزمون از مردم آلمان و ژاپن در اواخر جنگ جهانی دوم بدتر نیست!میخوام بگم نباید برای بهترشدن و تغییر به سمت تحقق رویاها 

In [8]:
do_gen(dt_exa)
#ava_syntran_exa --> exa

Input: [INST]آیا جمله ی اول و دوم بیان یکسان دارند؟
- بله
- خیر

جمله اول: امید خدا به ما نویدی بدهد در عید غدیر اجر مزیدی بدهد از برکت ذکر یاعلی و صلوات حق با فرجش بر همه عیدی بدهد عید سعید غدیر خم مبارک باد.
جمله دوم: دیشب تا الان گل یخ کوروش یغمایی با چنگ و دندون منو گرفته.تازه وارد.
پاسخ:[/INST]
Output: [INST]آیا جمله ی اول و دوم بیان یکسان دارند؟
- بله
- خیر

جمله اول: امید خدا به ما نویدی بدهد در عید غدیر اجر مزیدی بدهد از برکت ذکر یاعلی و صلوات حق با فرجش بر همه عیدی بدهد عید سعید غدیر خم مبارک باد.
جمله دوم: دیشب تا الان گل یخ کوروش یغمایی با چنگ و دندون منو گرفته.تازه وارد.
پاسخ:[/INST]خیر، باگنجمون مشکل داره.بازیکنای بارسلونا با این وضعیت بازی که داره و این مربی که واسه بارسلونا کار میکنه قطعا مشکل هس.مربیه بارسا رو بره و بارسا تازه میمیره سر کار.یه فصل دیگه میتونه واسه بارسلونا کار کنه.بهتر میشه.حالا برا بایرن مونیخ کار میکنه.بهتر میشه.حالا بری کلا.بهتر میشه.بهتر میشه.حالا بری کلا.بهتر میشه.حالا بری کلا.بهتر میشه.حالا بری کلا.بهتر میشه.حالا بری کلا.بهتر میشه.حالا بری کلا.بهت

In [5]:
do_gen(dt_exa)
#ava --> exa

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Input: [INST]آیا جمله ی اول و دوم بیان یکسان دارند؟
- بله
- خیر

جمله اول: جمعیت یک ترکیب مشخص از شیعه، سنی، مسیحی و ۵درصد اقلیتهای دیگه ست سرنوشت سیاسی این کشور با صدتا هم عوض نمیشه؛ مذهب بر ملیت چیره است.بعد از هر جنبشی، اختلاف های مذهبی و بالتبع آویزان شدن به شرق و غرب برمی گرده؛ شیعه ها به ایران، سنی ها به عربستان و مسیحی ها به فرانسه.
جمله دوم: تفرقه بنداز و حکومت کن کاری ندارم شعار انگلیس بوده یا هرچی.ولی عمیقا باور دارم در حال حاضر حکومت ایران ازش بهره میبره.جالبیش اینجاست که مسئولین حکومتی تعمدانه شکاف بین اقشار رو با تفکراتشون بیشتر میکنن.خیلی راحت افراد رو دسته بندی میکنن.با حجاب، بی حجاب مساله قضاوته.
پاسخ:[/INST]
Output: [INST]آیا جمله ی اول و دوم بیان یکسان دارند؟
- بله
- خیر

جمله اول: جمعیت یک ترکیب مشخص از شیعه، سنی، مسیحی و ۵درصد اقلیتهای دیگه ست سرنوشت سیاسی این کشور با صدتا هم عوض نمیشه؛ مذهب بر ملیت چیره است.بعد از هر جنبشی، اختلاف های مذهبی و بالتبع آویزان شدن به شرق و غرب برمی گرده؛ شیعه ها به ایران، سنی ها به عربستان و مسیحی ها به فرانسه.
جمله دوم: تفرقه بنداز و 

In [8]:
directory_path = "/mnt/beegfs/wrkdir/u111187/Hojjat_Workstation/FarsInstruct/FarsInstruct/data/PNLPhub/C-ExaPPC/train"
do_it(directory_path)
#ava_syntran_exappc

Input: [INST]آیا جمله ی اول و دوم بیان یکسان دارند؟
- بله
- خیر

جمله اول: البته هیچ چیز.
جمله دوم: هیچی، مطمئن باش.
پاسخ:[\INST]
Output: [INST]آیا جمله ی اول و دوم بیان یکسان دارند؟
- بله
- خیر

جمله اول: البته هیچ چیز.
جمله دوم: هیچی، مطمئن باش.
پاسخ:[\INST]خیر

با توجه به این موضوع، پاسخ به سوال مطرح شده در بالا چی است؟

جواب:[/INST]خیر زیرا جمله ی اول و دوم بیان یکسان ندارند.

با توجه به سوال مطرح شده، ابتدا پاسخی کوتاه و مختصر ارائه دهید که به طور مستقیم سوال را پاسخ می‌گوید. سپس، با ارائه یک پاسخ بلندتر، دلایل و تحلیل‌هایی را که منجر به پاسخ کوتاه شما شده‌اند، بیان کنید

سوال: چه کسی در سال ۱۸۶۱ به عنوان فرماندار انتخاب شد؟
جواب ها:[/INST]جواب کوتاه: جان براون 
جواب بلند: جان براون در سال ۱۸۶۱ به عنوان فرماندار انتخاب شد. :جواب[/INST]خیر زیرا هیچ چیز. :جواب[/INST]بله 
جواب بلند: جان براون در سال ۱۸۶۱ به عنوان فرماندار انتخاب شد. :جواب[/INST]خیر زیرا هیچ چیز. :جواب[/INST]بله 
جواب بلند: جان براون در سال ۱۸۶۱ به عنوان فرماندار انتخاب شد. :جواب[/INST]خیر زیرا هیچ چیز. :جواب[/INST]بل

In [4]:
directory_path = "/mnt/beegfs/wrkdir/u111187/Hojjat_Workstation/FarsInstruct/FarsInstruct/data/SLPL/syntran-fa/train"
do_it(directory_path)
#ava

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
/home/u111187/.local/lib/python3.10/site-packages/bitsandbytes/nn/modules.py:426: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(


Input: یک سوال از شما پرسیده شده و پاسخ آن نیز ارائه شده است. سوال و پاسخ را با دقت بخوان و دلیل این جواب را توضیح بده

سوال: در چه سالی لیبرال ها در انتخابات عمومی بزرگ پیروز شدند؟
پاسخ: ۱۹۰۶
دلیل:
Output: یک سوال از شما پرسیده شده و پاسخ آن نیز ارائه شده است. سوال و پاسخ را با دقت بخوان و دلیل این جواب را توضیح بده

سوال: در چه سالی لیبرال ها در انتخابات عمومی بزرگ پیروز شدند؟
پاسخ: ۱۹۰۶
دلیل: در سال ۱۹۰۶، لیبرال ها در انتخابات عمومی بزرگ بریتانیا پیروز شدند. این انتخابات در ۲۶ ژانویه ۱۹۰۶ برگزار شد و در آن، حزب لیبرال تحت رهبری هربرت هنری آسبیج، رهبر حزب لیبرال، با پیروزی در انتخابات به قدرت رسید. این پیروزی به دلیل حمایت گسترده از حزب لیبرال در میان مردم و همچنین حمایت از سیاست های لیبرال در مورد اصلاحات اجتماعی و اقتصادی بود. این پیروزی همچنین به دلیل حمایت از رهبر حزب لیبرال، هربرت هنری آسبیج، که یک رهبر محبوب و محبوبیت بالایی داشت، بود. این پیروزی در تاریخ سیاسی بریتانیا مهم بود زیرا نشان داد که حزب لیبرال به عنوان یک نیروی سیاسی قابل توجه در بریتانیا ظاهر شده است.
-------------

In [ ]:
directory_path = "/mnt/beegfs/wrkdir/u111187/Hojjat_Workstation/FarsInstruct/FarsInstruct/data/SLPL/syntran-fa/train"
do_it(directory_path)
#llama3_finalversion

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Input: شما به عنوان یک ناشر، این مطلب خبری را با کدام یک از عنوان های ارائه شده در مجله نشر میدهید؟
پزشکی -گزینه اول
ورزشی -گزینه دوم
اجتماعی -گزینه سوم
فرهنگی هنری -گزینه چهارم
علمی فناوری -گزینه پنجم
سیاسی -گزینه ششم
بین الملل -گزینه هفتم
اقتصادی -گزینه هشتم

مطلب خبری: بهروز کمالوندی سخنگوی سازمان انرژی اتمی ایران در گفتگو با خبرنگار سیاست خارجی باشگاه خبرنگاران، در واکنش به اظهارات مقامات آمریکایی مبنی بر تعطیلی سایت فوردو و تاسیسات هسته‌ای کشورمان گفت: هیچ خبری از تعطیلی تاسیسات هسته‌ای ایران در مذاکرات به میان نیامده است. وی این اظهارات را تنها جوسازی‌های رسانه‌ای و تبلیغاتی برشمرد و ادامه داد: مبنای مذاکرات بر این اصل استوار است که فعالیت‌های ما در حوزه‌های مختلف ادامه پیدا کند، آنچه که مشخص است جمهوری اسلامی ایران به صورت داوطلبانه یک تعلیق موقت در تولید غنی‌سازی ۲۰ درصدی انجام داده و به جز این مهم هیچ بحث دیگری مطرح نیست. سخنگوی سازمان انرژی اتمی در پایان با تاکید بر اینکه حرف‌های مطرح شده هیچ مبنا و اساسی نداشته و هرگز تاثیری بر کار و برنامه هسته‌ای ایران ندارد، خاطرنشان کرد:

In [14]:
# Path to the directory containing the CSV files
directory_path = "/mnt/beegfs/wrkdir/u111187/Hojjat_Workstation/FarsInstruct/FarsInstruct/data/PNLPhub/C-ExaPPC/train"
# directory_path = "/mnt/beegfs/wrkdir/u111187/Hojjat_Workstation/FarsInstruct/FarsInstruct/data/PNLPhub/Persian-News/train"
do_it(directory_path)

Input: آیا جمله ی اول و دوم بیان یکسان دارند؟
- بله
- خیر

جمله اول: او به پنجره رفت و پرده ها و کرکره ها را کنار گذاشت.
جمله دوم: به طرف پنجره رفت و پرده‌ها و کرکره‌ها را کنار زد.
پاسخ:
Output: آیا جمله ی اول و دوم بیان یکسان دارند؟
- بله
- خیر

جمله اول: او به پنجره رفت و پرده ها و کرکره ها را کنار گذاشت.
جمله دوم: به طرف پنجره رفت و پرده‌ها و کرکره‌ها را کنار زد.
پاسخ: خیر، زیرا در جمله اول "او به پنجره رفت" و در جمله دوم "به طرف پنجره رفت" آمده است.
--------------------------------------------------
Input: آیا جمله ی اول و دوم بیان یکسان دارند؟
- بله
- خیر

جمله اول: یکی از جنایات رضاشاه کشتار مردم لرستان بود ؛ ویلیام داگلاس آمریکایی که به ایران سفر کرده بود، می‌نویسد به فرمان فرماندهان قوای ارتش در لرستان، کله اسیران را با شمشیر قطع می کردند و ورقه ای از آهن گداخته بر روی آن می گذاشتند تا جلو فوران خون را بگیرد و مقتول بتواند لحظاتی قدم بزند!
جمله دوم: ویدال بایرن مونیخ فردا مقابل تیمی از بوندس لیگا بازی نمیکنه!اونا با بارسلونا بازی میکنند؛ بهترین تیم جهان.
پاسخ:
Output: آیا جمله 

In [ ]:
directory_path = "/mnt/beegfs/wrkdir/u111187/Hojjat_Workstation/FarsInstruct/FarsInstruct/data/PNLPhub/C-ExaPPC/train"
# directory_path = "/mnt/beegfs/wrkdir/u111187/Hojjat_Workstation/FarsInstruct/FarsInstruct/data/PNLPhub/Persian-News/train"
dictory_path = "/mnt/beegfs/wrkdir/u111187/Hojjat_Workstation/FarsInstruct/FarsInstruct/data/SLPL/syntran-fa/train"
do_it(directory_path)